In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
home_path = '/gws/nopw/j04/duicv/yuansun/'

In [3]:
sensor_list = ['MOD-PM-00454', 'MOD-PM-00459', 'MOD-PM-00461']
sensor_datapath = home_path + 'manchester/sensor/analyse_variable/temperature/tem_rh_hourly/'
meta_data = pd.read_csv(home_path + 'manchester/sensor/meta_data/meta_data_final_for_analyse.csv')
supersite_data = pd.read_csv(home_path + 'manchester/supersite/hourly_data/final_supersite_merged.csv')
supersite_data['Datetime'] = pd.to_datetime(supersite_data['Datetime'])
supersite_data.rename(columns={'Datetime': 'timestamp'}, inplace=True)
sliced_supersite_data = supersite_data[(supersite_data['timestamp'] >= '2022-07-18 15:00:00') & (supersite_data['timestamp'] <= '2024-06-07 20:00:00')]
sliced_supersite_data = sliced_supersite_data[['timestamp', 'TBOT', 'RH']].rename(columns={'TBOT': 'temp_hourly_avg', 'RH': 'rh_hourly_avg'}).reset_index(drop=True)
bias_list = []
for sensor in sensor_list:
    sensor_data = pd.read_csv(sensor_datapath + sensor + '.csv', parse_dates=['timestamp'])
    bias = pd.DataFrame()
    bias['timestamp'] = sensor_data['timestamp']
    bias['temp_hourly_avg'] = (sensor_data['temp_hourly_avg'] - sliced_supersite_data['temp_hourly_avg']) / sliced_supersite_data['temp_hourly_avg']
    bias['rh_hourly_avg'] = (sensor_data['rh_hourly_avg'] - sliced_supersite_data['rh_hourly_avg']) / sliced_supersite_data['rh_hourly_avg']
    bias.set_index('timestamp', inplace=True)
    # remove the outliers
    bias['temp_hourly_avg'] = bias['temp_hourly_avg'].where((bias['temp_hourly_avg'] <= 15)&(bias['temp_hourly_avg'] >= -15), np.nan)
    #print(bias[bias['temp_hourly_avg']<-15])
    #bias['hour'] = bias['timestamp'].dt.hour
    #bias['month'] = bias['timestamp'].dt.month
    #bias = (sensor_data.set_index('timestamp') - sliced_supersite_data) / sliced_supersite_data
    bias_list.append(bias)
all_biases = pd.concat(bias_list, axis=1)
# control the qulity of all_biases
mean_bias_tem = pd.DataFrame(all_biases['temp_hourly_avg'].mean(axis=1), columns=['mean_bias'])
mean_bias_rh = pd.DataFrame(all_biases['rh_hourly_avg'].mean(axis=1), columns=['mean_bias'])
mean_bias_tem['hour'] = mean_bias_tem.index.hour
mean_bias_rh['hour'] = mean_bias_rh.index.hour
mean_bias_tem['month'] = mean_bias_tem.index.month
mean_bias_rh['month'] = mean_bias_rh.index.month
monthly_mean_bias_tem = mean_bias_tem.groupby(['month', 'hour']).mean()
monthly_mean_bias_rh = mean_bias_rh.groupby(['month', 'hour']).mean()


In [27]:
print(sensor_data['rh_hourly_avg'].min(), sensor_data['rh_hourly_avg'].max())

14.82611111111111 91.54666666666668


In [29]:
print(sensor_data['temp_hourly_avg'].min(), sensor_data['temp_hourly_avg'].max())

-5.938166666666667 42.83533333333333


In [22]:
check = monthly_mean_bias_rh.reset_index()
check[check['hour']==18]

,month,hour,mean_bias
18,1,18,-0.102477
42,2,18,-0.110518
66,3,18,-0.116940
90,4,18,-0.145512
114,5,18,-0.179767
138,6,18,-0.190531
162,7,18,-0.172164
186,8,18,-0.172189
210,9,18,-0.127869
234,10,18,-0.108885


In [6]:
print(all_biases['rh_hourly_avg'].min(), all_biases['rh_hourly_avg'].max())

rh_hourly_avg   -0.411717
rh_hourly_avg   -0.400747
rh_hourly_avg   -0.435157
dtype: float64 rh_hourly_avg   -0.016407
rh_hourly_avg   -0.018189
rh_hourly_avg   -0.009977
dtype: float64


In [42]:
for sensor in meta_data['sn']:
    if sensor not in sensor_list:
        sensor_data = pd.read_csv(sensor_datapath + sensor + '.csv')
        sensor_data['timestamp'] = pd.to_datetime(sensor_data['timestamp'])
        sensor_data['hour'] = sensor_data['timestamp'].dt.hour
        sensor_data['month'] = sensor_data['timestamp'].dt.month
        merged_data = pd.merge(sensor_data, monthly_mean_bias_rh, on=['month', 'hour'], how='left')
        merged_data = pd.merge(merged_data, monthly_mean_bias_tem, on=['month', 'hour'], how='left')
        merged_data = merged_data.rename(columns={'mean_bias_x': 'rh_mean_bias', 'mean_bias_y': 'temp_mean_bias'})
        merged_data['rh_hourly_avg_corrected'] = merged_data['rh_hourly_avg'] / (1 + merged_data['rh_mean_bias'])
        merged_data['temp_hourly_avg_corrected'] = merged_data['temp_hourly_avg'] / (1 + merged_data['temp_mean_bias'])
        data_for_save = merged_data[['timestamp', 'rh_hourly_avg_corrected', 'temp_hourly_avg_corrected']]
        # if relative humidity is over 100%, set it to 100%
        data_for_save.loc[data_for_save['rh_hourly_avg_corrected']>100, 'rh_hourly_avg_corrected']= 100
        data_for_save.to_csv('./adjusted_sensor_data/' + sensor + '.csv', index=False)


In [43]:
print(data_for_save['rh_hourly_avg_corrected'].min(), data_for_save['rh_hourly_avg_corrected'].max())
print(data_for_save['temp_hourly_avg_corrected'].min(), data_for_save['temp_hourly_avg_corrected'].max())

27.959354339075162 100.0
0.45621224941974764 29.399030743187826
